In [ ]:
%matplotlib inline

#Calculations
import numpy as np

#Plots
import matplotlib as mpl
from matplotlib import pyplot as plt

# Unit conversions and constant definitions
from astropy import constants as const
from astropy import units as u

# Optimizer:
from scipy.optimize import minimize

%pylab inline

# Make inline plots larger.
pylab.rcParams['figure.figsize'] = (14, 8)

# TeX rendering
mpl.rc('text', usetex = True)
mpl.rc('font', family = 'serif')

# Make axis tick labels larger
mpl.rcParams['xtick.labelsize'] = 15
mpl.rcParams['ytick.labelsize'] = 15

#Generic object class for readability:
class Object:
    def __init__(self, **kwds):
        self.__dict__.update(kwds)

In [ ]:
def eccentricity_derivative(m1, m2, a, e):
    top_left = -304*const.G**3.*m1*m2*(m1+m2)
    bot_left = 15*a**4.*const.c**5.
    top_right = e*(1+121.*e**2./304.)
    bot_right = (1-e**2.)**(5./2)
    
    derivative = top_left*top_right/bot_left/bot_right
    return derivative

In [ ]:
def radius_derivative(m1, m2, a, e):
    top_left = -64*const.G**3.*m1*m2*(m1+m2)
    bot_left = 5*a**3.*const.c**5.
    top_right = 1+73.*e**2./24 + 37.*e**4./96
    bot_right = (1-e**2.)**(7./2)
    
    derivative = top_left*top_right/bot_left/bot_right
    return derivative

In [ ]:
def run_merger_sim(m1, m2, a0, e0, max_steps=10000):
    a = a0
    e = e0
    step_size = 1*u.s
    for step in range(10000):
        e_dot = eccentricity_derivative(m1, m2, a, e)
        a_dot = radius_derivative(m1, m2, a, e)
        if np.abs(e_dot*step_size/e) > 0.01 or \
                np.abs(a_dot*step_size/a) > 0.01:
            step_size /= 10.
        elif np.abs(e_dot*step_size/e) < 0.01 and \
                np.abs(a_dot*step_size/a) < 0.01:
            step_size *= 10.
        e = e + e_dot*step_size
        a = a + a_dot*step_size
        #print("a, e, step_size", a.to('m').value, e.value, step_size.to("yr").value)
        if e < 1e-2:
            return 0
        if a.to('km').value < 24:
            return e

In [ ]:
for _ in range(2):
    m1 = 10*np.random.random()*const.M_sun
    m2 = 10*np.random.random()*const.M_sun
    a = 10*np.random.random()*const.au
    e = np.random.random()
    run_merger_sim(m1, m2, a, e)
    print m1.to("M_sun").value, ",",
    print m2.to("M_sun").value, ",", a.to("km").value, ",", e